# Generate flex_ddG DMS files

This script produces the input mutation files for a flexddG DMS run.

In [2]:
# Load libraries
import numpy as np
import pandas as pd
import matplotlib
import csv
import os
import sys
import subprocess
from Bio.PDB import PDBParser, PDBIO
import glob
from Bio import SeqIO
import re
import json
from collections import OrderedDict
import seaborn as sns
import shutil
import json

## Prepare systematic DMS runs for all possible mutations

In [5]:
# Load the data with all the mutations we can simulate
foldx_effects = pd.read_csv('../../Data/Mutational_effects/sciadv.add9109_table_s3.tsv', 
                               sep = '\t')
foldx_effects

<ipython-input-5-2c12c97b9e89>:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  foldx_effects = pd.read_csv('../../Data/Data_Cisneros2023/sciadv.add9109_table_s3_with_HETddGs.tsv',


,Position,WT_Residue,Residue,Timepoint,Arabinose,TMP,mean_sel_coeff,sd_sel_coeff,sem_sel_coeff,num_samples,...,retention_coefficient_hfba_browne,retention_coefficient_ph2.1_meek,retention_coefficient_ph7.4_meek,retention_coefficient_tfa_browne,total_beta_strand_lifson,transmembrane_tendency_zhao,levy_propensity,Mean_ddG_int_HET_A_B,Mean_ddG_int_HET_A_C,Mean_ddG_int_HET_A_D
0,2,E,*,10,0.010,0,0.003049,0.000030,0.000021,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,E,*,10,0.010,10,-0.069929,0.028430,0.012714,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,E,*,10,0.025,0,-0.016385,0.004606,0.003257,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,E,*,10,0.025,10,-0.111875,0.025297,0.011313,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,E,*,10,0.050,0,-0.007821,0.000403,0.000285,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16145,78,N,Y,10,0.050,10,0.021275,0.010312,0.007292,2,...,6.6,9.8,5.3,11.6,0.93,2.11,1.1499,0.0,0.0,0.0
16146,78,N,Y,10,0.200,0,-0.009799,0.006995,0.004946,2,...,6.6,9.8,5.3,11.6,0.93,2.11,1.1499,0.0,0.0,0.0
16147,78,N,Y,10,0.200,10,0.017033,0.005412,0.002420,5,...,6.6,9.8,5.3,11.6,0.93,2.11,1.1499,0.0,0.0,0.0
16148,78,N,Y,10,0.400,0,-0.003240,0.009266,0.006552,2,...,6.6,9.8,5.3,11.6,0.93,2.11,1.1499,0.0,0.0,0.0


In [6]:
## Remove positions from the disordered region and remove unnecessary columns
foldx_effects_noNA = foldx_effects[['Position', 'WT_Residue', 'Residue', 'Mean_ddG_int_HM_A_C']].dropna(subset = ['Mean_ddG_int_HM_A_C'])
foldx_effects_noNA

,Position,WT_Residue,Residue,Mean_ddG_int_HM_A_C
4210,22,A,A,0.0
4211,22,A,A,0.0
4212,22,A,A,0.0
4213,22,A,A,0.0
4214,22,A,A,0.0
...,...,...,...,...
16145,78,N,Y,0.0
16146,78,N,Y,0.0
16147,78,N,Y,0.0
16148,78,N,Y,0.0


Start with the run for NADPH

In [8]:
already_seen = []

## New way to organize the output
main_folder = '/path/output/folder'
os.makedirs(main_folder, exist_ok = True)

tetramer_path = main_folder

chains_move_tetramer = 'A,B,C,D'
count_folders = 0

for index, line in foldx_effects_noNA.iterrows():

    ## Read the mutation
    mut1 = line['WT_Residue'] + str(line['Position']) + line['Residue']
    
    mut1_wt = mut1[0]
    mut1_pos = mut1[1:-1]
    mut1_res = mut1[-1]
        
    ## Prepare files for mutation 1 if it has not been done already and it is not in the disordered region
    if not mut1 in already_seen and int(mut1_pos) >= 21:
    
        count_folders += 1
        
        ## Create the directories for this pair of mutations
        out_path_tetramers_mut = os.path.join(tetramer_path, mut1)
        if not os.path.exists(out_path_tetramers_mut):
            os.makedirs(out_path_tetramers_mut)

        outfile_tetramers_handle = open(os.path.join(out_path_tetramers_mut, 'mutation_list.txt'), 'w')

        ## Rosetta flex ddG input format: 
        # A.F.118.W,B.F.118.W A
        # (chain,WT_res,pos,new_res,chain,WT_res,pos,new_res,...[space][chains_to_move])

        #### Generate the tetramer combinations ####
        ## Tetramer 1 (mut1 in all four chains)
        new_line =  '.'.join(['A', mut1_wt, mut1_pos, mut1_res]) + ','
        new_line += '.'.join(['B', mut1_wt, mut1_pos, mut1_res]) + ','
        new_line += '.'.join(['C', mut1_wt, mut1_pos, mut1_res]) + ','
        new_line += '.'.join(['D', mut1_wt, mut1_pos, mut1_res])
        new_line += ' ' + chains_move_tetramer
        outfile_tetramers_handle.write(new_line + '\n')

        ## Tetramer 2 (mut1 in A; WT in B,C,D)
        new_line =  '.'.join(['A', mut1_wt, mut1_pos, mut1_res])
        new_line += ' ' + chains_move_tetramer
        outfile_tetramers_handle.write(new_line + '\n')

        ## Tetramer 3 (mut1 in B,C,D; WT in A)
        new_line =  '.'.join(['B', mut1_wt, mut1_pos, mut1_res]) + ','
        new_line += '.'.join(['C', mut1_wt, mut1_pos, mut1_res]) + ','
        new_line += '.'.join(['D', mut1_wt, mut1_pos, mut1_res])
        new_line += ' ' + chains_move_tetramer
        outfile_tetramers_handle.write(new_line + '\n')
        
        ## Tetramer 4 (mut1 in A,C; WT in B,D)
        new_line =  '.'.join(['A', mut1_wt, mut1_pos, mut1_res]) + ','
        new_line += '.'.join(['C', mut1_wt, mut1_pos, mut1_res])
        new_line += ' ' + chains_move_tetramer
        outfile_tetramers_handle.write(new_line + '\n')

        ## Tetramer 5 (mut1 in A,D; WT in B,C)
        new_line =  '.'.join(['A', mut1_wt, mut1_pos, mut1_res]) + ','
        new_line += '.'.join(['D', mut1_wt, mut1_pos, mut1_res])
        new_line += ' ' + chains_move_tetramer
        outfile_tetramers_handle.write(new_line + '\n')
        
        ## Tetramer 6 (mut1 in A,B; WT in C,D)
        new_line =  '.'.join(['A', mut1_wt, mut1_pos, mut1_res]) + ','
        new_line += '.'.join(['B', mut1_wt, mut1_pos, mut1_res])
        new_line += ' ' + chains_move_tetramer
        outfile_tetramers_handle.write(new_line + '\n')
        
        ## Tetramer 7 would be WT in all four subunits, which we don't need
        
        ## Add mutations to already_seen
        already_seen.append(mut1)

        ## Close the handles
        outfile_tetramers_handle.close()


Repeat for DHF binding to the (DfrB1 tetramer + NADPH)

In [11]:
already_seen = []

## New way to organize the output
main_folder = '/path/output/folder'
os.makedirs(main_folder, exist_ok = True)

tetramer_path = main_folder

chains_move_tetramer = 'A,B,C,D,E'

count_folders = 0

for index, line in foldx_effects_noNA.iterrows():

    ## Read the mutation
    mut1 = line['WT_Residue'] + str(line['Position']) + line['Residue']
    
    mut1_wt = mut1[0]
    mut1_pos = mut1[1:-1]
    mut1_res = mut1[-1]
        
    ## Prepare files for mutation 1 if it has not been done already and it is not in the disordered region
    if not mut1 in already_seen and int(mut1_pos) >= 21:
    
        # print(mut1, mut1_wt, mut1_pos, mut1_res)
        
        count_folders += 1

        ## Create the directories for this pair of mutations
        out_path_tetramers_mut = os.path.join(tetramer_path, mut1)
        if not os.path.exists(out_path_tetramers_mut):
            os.makedirs(out_path_tetramers_mut)

        outfile_tetramers_handle = open(os.path.join(out_path_tetramers_mut, 'mutation_list.txt'), 'w')

        ## Rosetta flex ddG input format: 
        # A.F.118.W,B.F.118.W A
        # (chain,WT_res,pos,new_res,chain,WT_res,pos,new_res,...[space][chains_to_move])

        #### Generate the tetramer combinations ####
        ## Tetramer 1 (mut1 in all four chains)
        new_line =  '.'.join(['A', mut1_wt, mut1_pos, mut1_res]) + ','
        new_line += '.'.join(['B', mut1_wt, mut1_pos, mut1_res]) + ','
        new_line += '.'.join(['C', mut1_wt, mut1_pos, mut1_res]) + ','
        new_line += '.'.join(['D', mut1_wt, mut1_pos, mut1_res])
        new_line += ' ' + chains_move_tetramer
        outfile_tetramers_handle.write(new_line + '\n')

        ## Tetramer 2 (mut1 in A; WT in B,C,D)
        new_line =  '.'.join(['A', mut1_wt, mut1_pos, mut1_res])
        new_line += ' ' + chains_move_tetramer
        outfile_tetramers_handle.write(new_line + '\n')

        ## Tetramer 3 (mut1 in B,C,D; WT in A)
        new_line =  '.'.join(['B', mut1_wt, mut1_pos, mut1_res]) + ','
        new_line += '.'.join(['C', mut1_wt, mut1_pos, mut1_res]) + ','
        new_line += '.'.join(['D', mut1_wt, mut1_pos, mut1_res])
        new_line += ' ' + chains_move_tetramer
        outfile_tetramers_handle.write(new_line + '\n')
        
        ## Tetramer 4 (mut1 in A,C; WT in B,D)
        new_line =  '.'.join(['A', mut1_wt, mut1_pos, mut1_res]) + ','
        new_line += '.'.join(['C', mut1_wt, mut1_pos, mut1_res])
        new_line += ' ' + chains_move_tetramer
        outfile_tetramers_handle.write(new_line + '\n')

        ## Tetramer 5 (mut1 in A,D; WT in B,C)
        new_line =  '.'.join(['A', mut1_wt, mut1_pos, mut1_res]) + ','
        new_line += '.'.join(['D', mut1_wt, mut1_pos, mut1_res])
        new_line += ' ' + chains_move_tetramer
        outfile_tetramers_handle.write(new_line + '\n')
        
        ## Tetramer 6 (mut1 in A,B; WT in C,D)
        new_line =  '.'.join(['A', mut1_wt, mut1_pos, mut1_res]) + ','
        new_line += '.'.join(['B', mut1_wt, mut1_pos, mut1_res])
        new_line += ' ' + chains_move_tetramer
        outfile_tetramers_handle.write(new_line + '\n')
        
        ## Tetramer 7 would be WT in all four subunits, which we don't need
        
        ## Add mutations to already_seen
        already_seen.append(mut1)

        ## Close the handles
        outfile_tetramers_handle.close()

    